# Project 3 LENET-5 CNN

Ashton Cole

AVC687

COE 379L: Software Design for Responsive Intelligent Systems

## Description

This notebook builds a LENET-5 Convolutional Neural Network.

In [1]:
import tensorflow as tf
from tensorflow.keras import optimizers, Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, AveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

2024-04-09 23:13:43.514435: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 23:13:43.555671: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 23:13:43.555734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 23:13:43.557760: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 23:13:43.566388: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 23:13:43.567869: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Preprocessing

In [2]:
validation_split = 0.2 # Validation split
batch_size = 32
seed = 123

image_height = 128
image_width = 128

train_dir = 'data/split/train'
test_dir = 'data/split/test'

rescale = Rescaling(scale=1.0/255)

data_train, data_validation = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=validation_split,
    subset='both',
    seed=seed,
    image_size=(image_height, image_width),
    batch_size=batch_size)
data_test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed=seed,
    image_size=(image_height, image_width),
    batch_size=batch_size)

data_train_scaled = data_train.map(lambda image, label: (rescale(image), label))
data_validation_scaled = data_validation.map(lambda image, label: (rescale(image), label))
data_test_scaled = data_test.map(lambda image, label: (rescale(image), label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.
Found 4265 files belonging to 2 classes.


## Model Construction

In [3]:
image_length = image_height * image_width * 3

model = Sequential()
# C1
model.add(Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
# S2
model.add(AveragePooling2D(pool_size=(2, 2)))
# C3
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
# F4
model.add(Flatten())
# F5
model.add(Dense(100, activation='relu'))
# F6
model.add(Dense(84, activation='relu'))
# Output
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d (Average  (None, 63, 63, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 16)        880       
                                                                 
 flatten (Flatten)           (None, 59536)             0         
                                                                 
 dense (Dense)               (None, 100)               5953700   
                                                                 
 dense_1 (Dense)             (None, 84)                8484      
                                                        

## Model Compilation and Fitting

In [4]:
optimizer=optimizers.RMSprop(learning_rate=1e-4)
loss='sparse_categorical_crossentropy'
metrics=['accuracy']

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

model.fit(
    x=data_train_scaled,
    epochs=5,
    validation_data=data_validation_scaled,
    verbose=2,
    batch_size=batch_size)

Epoch 1/5
427/427 - 45s - loss: 0.5655 - accuracy: 0.7297 - val_loss: 0.4517 - val_accuracy: 0.8042 - 45s/epoch - 105ms/step
Epoch 2/5
427/427 - 42s - loss: 0.4360 - accuracy: 0.8271 - val_loss: 0.3820 - val_accuracy: 0.8678 - 42s/epoch - 98ms/step
Epoch 3/5
427/427 - 29s - loss: 0.4004 - accuracy: 0.8427 - val_loss: 0.3620 - val_accuracy: 0.8725 - 29s/epoch - 68ms/step
Epoch 4/5
427/427 - 29s - loss: 0.3812 - accuracy: 0.8486 - val_loss: 0.5358 - val_accuracy: 0.7543 - 29s/epoch - 68ms/step
Epoch 5/5
427/427 - 29s - loss: 0.3633 - accuracy: 0.8556 - val_loss: 0.3670 - val_accuracy: 0.8461 - 29s/epoch - 67ms/step


## Model Testing

In [5]:
test_loss, test_accuracy = model.evaluate(data_test_scaled, verbose=0)
test_accuracy

0.8415005803108215